In [1]:
import pandas as pd
from pathlib import Path

In [2]:
FILE_PATH = Path('data').joinpath('STO Report 01.09.21-19.12.21.XLSX')

In [5]:
FILE_PATH

WindowsPath('data/STO Report 01.09.21-19.12.21.XLSX')

In [19]:
REQUIRED_COLUMNS = [
    'STO No.', 'STO Status', 'Receiving Plant', 
    'Item Description', 'Item SKU Qty', 'SKU UoM', 
    'Delivery No', 'PGI Quantity', 'PGI UoM','PGI Date'
]

COLUMN_CLEANED = [item.lower().replace(" ", "_").replace(".", "").replace("/", "") for item in REQUIRED_COLUMNS]

COLUMN_SCHEMA = dict(zip(REQUIRED_COLUMNS, COLUMN_CLEANED))

WEEKLY_RPD_REQUIRED_COLUMNS = ['STO No.', 'Item', 'Quantity (kg)', 'Production', 'FG']

WEEKLY_RPD_REQUIRED_COLUMNS_CLEANED = [
    item.lower()
    .replace(" ", "_")
    .replace(".", "")
    .replace("/", "")
    .replace("(", "")
    .replace(")", "")
    
    for item in WEEKLY_RPD_REQUIRED_COLUMNS
]

WEEKLY_PRD_SCHEMA = dict(zip(WEEKLY_RPD_REQUIRED_COLUMNS, WEEKLY_RPD_REQUIRED_COLUMNS_CLEANED))

In [4]:
# temp_df = pd.read_excel(FILE_PATH, sheet_name="Weekly Prod. Sche. 12.12-18.12", skiprows=2)

In [6]:
def read_to_dataframe(
    path: str, 
    required_column: list, 
    column_schema: dict, 
    sheet_name: str
) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=sheet_name)
    df = df[required_column]
    df = df.rename(columns=column_schema)
    df = df[(~df["sto_no"].isnull()) & (~df.item_description.isnull()) & (~df.receiving_plant.isnull())]
    return df

In [38]:
def read_weekly_prod(
    path: str, 
    required_column: list, 
    column_schema: dict, 
    sheet_name: str,
    skip_row=2
) -> pd.DataFrame:
    df = pd.read_excel(path, sheet_name=sheet_name, skiprows=skip_row)
    df = df[required_column]
    df = df.rename(columns=column_schema)
    df = df[~df["sto_no"].isnull()]
    df = df[df["sto_no"] != "Total"]
    df["sto_no"] = df["sto_no"].astype(str)
    return df

In [8]:
RAW_DF = read_to_dataframe(
    path=FILE_PATH, 
    required_column=REQUIRED_COLUMNS, 
    column_schema=COLUMN_SCHEMA, 
    sheet_name="Sheet1"
)

In [54]:
WEEKLY_PROD = read_weekly_prod(
    path=FILE_PATH, 
    required_column=WEEKLY_RPD_REQUIRED_COLUMNS, 
    column_schema=WEEKLY_PRD_SCHEMA, 
    sheet_name="Weekly Prod. Sche. 12.12-18.12",
)

In [55]:
WEEKLY_PROD.tail()

,sto_no,item,quantity_kg,production,fg
22,4500025774,Wrap-No.1 Orange Sandwch Crm Biscuit 30g,1000.0,NaN,NaN
23,4500025775,Wrap-No.1 Pineapple Sandwh Crm Bisct 30g,1000.0,NaN,NaN
24,4500025916,Wrap-Fresh Sugar Crush Biscuit 24g,2000.0,1889.0,NaN
25,4500025916,"Wrap-Fresh Butter Bun 45g,L200XW170MM",2500.0,2116.0,NaN
35,4500026490,Label-Super Fresh Fort Soybean Oil 1L,1200.0,1200.0,262.0


In [44]:
# for item in WEEKLY_PROD["sto_no"].to_list():
#     print(item, type(item), int(item))

In [59]:
def create_temp_sto_no(df: pd.DataFrame) -> pd.DataFrame:
    df["temp"] = df["sto_no"].astype('int64').astype('str')
    return df

In [61]:
def create_uid(df: pd.DataFrame, col1: str, col2: str) -> pd.DataFrame:
    df = create_temp_sto_no(df)
    df["uid"] = df[col1] + " " + df[col2]
    df = df.drop("temp", axis=1)
    return df

In [71]:
def make_pgi_qty_df(df: pd.DataFrame) -> pd.DataFrame:
    selected_columns = [
        'sto_no', 'item_description', 'receiving_plant', 
        'delivery_no', 'pgi_date', 'pgi_quantity', 'pgi_uom'
    ]
    _df = df[selected_columns].copy()
    _df = create_temp_sto_no(_df)
    _df = create_uid(df=_df, col1="temp", col2="item_description")
    new_col_list = [
        'uid', 'sto_no', 'item_description', 'receiving_plant', 
        'delivery_no', 'pgi_date', 'pgi_quantity', 'pgi_uom'
    ]
    _df = _df[new_col_list]
    return _df

In [72]:
def make_weekly_prod_df(df: pd.DataFrame) -> pd.DataFrame:
    _df = create_temp_sto_no(df)
    _df = create_uid(_df, "temp", "item")
    new_col_list = ["uid", "sto_no", "item", "quantity_kg", "production", "fg"]
    _df = _df[new_col_list]
    return _df

In [73]:
PGI_QTY_DF = make_pgi_qty_df(RAW_DF)

In [74]:
WKL_DLV_DF = make_weekly_prod_df(WEEKLY_PROD)

In [82]:
def merge_pgi_weekly(df1: pd.DataFrame, df2: pd.DataFrame, key: str, how: str):
    _df = df1.merge(df2, on=key, how=how)
    return _df

In [83]:
def has_gum_text(text: str):
    if "gum" in text.lower():
        return True
    return False

In [81]:
# def x_square(x):
#     return x*2

# x_square_one_line = lambda x: x*x

In [93]:
def make_summary_df(df: pd.DataFrame) -> pd.DataFrame:
    selected_columns = [
        "sto_no", "item_description", 
        "receiving_plant", "sku_uom", 
        "item_sku_qty", "pgi_quantity"
    ]
    _df = df[selected_columns]
    _df = _df[_df["sku_uom"] != "PC"]
    _df = _df[~df["item_description"].apply(lambda x: has_gum_text(x))]
    _df = create_uid(df=_df, col1="temp", col2="item_description")
    _df = (_df
           .groupby(["uid", "sto_no", "item_description", "receiving_plant", "sku_uom", "item_sku_qty"])
           .agg(pgi_quantity=("pgi_quantity", "sum"))
           .reset_index()
          )
    _df["pending_qty"] = _df["item_sku_qty"] - _df["pgi_quantity"]
    return _df.copy()

In [94]:
SUMMARY_DF = make_summary_df(RAW_DF)

C:\Users\ESHAYA~1.HAS\AppData\Local\Temp/ipykernel_14952/2831373945.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  _df = _df[~df["item_description"].apply(lambda x: has_gum_text(x))]


In [95]:
SUMMARY_DF.head()

,uid,sto_no,item_description,receiving_plant,sku_uom,item_sku_qty,pgi_quantity,pending_qty
0,4500023106 Wrapper-Fresh Refined Sugar Pouch 1KG,4.500023e+09,Wrapper-Fresh Refined Sugar Pouch 1KG,5101.0,KG,15000.0,15000,0.0
1,4500023106 Wrapper-Fresh Refined Sugar Pouch 500G,4.500023e+09,Wrapper-Fresh Refined Sugar Pouch 500G,5101.0,KG,1500.0,1500,0.0
2,4500023106 Wrapper-No.1 Refined Sugar Pouch 1KG,4.500023e+09,Wrapper-No.1 Refined Sugar Pouch 1KG,5101.0,KG,1500.0,1500,0.0
3,4500023106 Wrapper-No.1 Refined Sugar Pouch 500G,4.500023e+09,Wrapper-No.1 Refined Sugar Pouch 500G,5101.0,KG,500.0,488,12.0
4,4500023244 Wrap-No.1 Pusti Protidin Milk 15g,4.500023e+09,Wrap-No.1 Pusti Protidin Milk 15g,3001.0,KG,5100.0,5100,0.0


In [96]:
WKL_SUMM_MERGED = merge_pgi_weekly(SUMMARY_DF, WKL_DLV_DF, "uid", "left")

In [97]:
WKL_SUMM_MERGED.head()

,uid,sto_no_x,item_description,receiving_plant,sku_uom,item_sku_qty,pgi_quantity,pending_qty,sto_no_y,item,quantity_kg,production,fg
0,4500023106 Wrapper-Fresh Refined Sugar Pouch 1KG,4.500023e+09,Wrapper-Fresh Refined Sugar Pouch 1KG,5101.0,KG,15000.0,15000,0.0,NaN,NaN,NaN,NaN,NaN
1,4500023106 Wrapper-Fresh Refined Sugar Pouch 500G,4.500023e+09,Wrapper-Fresh Refined Sugar Pouch 500G,5101.0,KG,1500.0,1500,0.0,NaN,NaN,NaN,NaN,NaN
2,4500023106 Wrapper-No.1 Refined Sugar Pouch 1KG,4.500023e+09,Wrapper-No.1 Refined Sugar Pouch 1KG,5101.0,KG,1500.0,1500,0.0,NaN,NaN,NaN,NaN,NaN
3,4500023106 Wrapper-No.1 Refined Sugar Pouch 500G,4.500023e+09,Wrapper-No.1 Refined Sugar Pouch 500G,5101.0,KG,500.0,488,12.0,NaN,NaN,NaN,NaN,NaN
4,4500023244 Wrap-No.1 Pusti Protidin Milk 15g,4.500023e+09,Wrap-No.1 Pusti Protidin Milk 15g,3001.0,KG,5100.0,5100,0.0,NaN,NaN,NaN,NaN,NaN


In [98]:
WRITE_PATH = Path("data").joinpath("merged.XLSX")

In [99]:
WKL_SUMM_MERGED.to_excel(WRITE_PATH, index=False)